In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from stop_words import get_stop_words
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleanedfakenews/cleaned.csv
/kaggle/input/fakenews/news_cleaned_2018_02_13.csv


In [5]:
train = pd.read_csv('/kaggle/input/fakenews/news_cleaned_2018_02_13.csv')

/tmp/ipykernel_34/1063975546.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('/kaggle/input/fakenews/news_cleaned_2018_02_13.csv')


In [6]:
train

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,NaN,[''],THE UNIVERSE ceases to exist when we are not l...,NaN,NaN,NaN
1,1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"Unfortunately, he hasn’t yet attacked her for ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN
2,2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN
3,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN
4,4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“The time has come to cut off the tongues of t...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",NaN,[''],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1828413,8081,1056770,unz.com,bias,https://www.unz.com/gnxp/get-off-your-ass-and-...,I was recently directed to this panegyric on W...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Get off your ass and do this study: Introducto...,"Agnostic Gnxp, Razib Khan, John Harrison Sims,...",NaN,[''],NaN,"9/11 Articles, American Pravda Articles, Race/...",NaN,NaN
1828414,8082,1056771,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974STA...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1974STATE092127,NaN,NaN,[''],NaN,View Tags,NaN,NaN
1828415,8083,1056772,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974LAP...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1974LAPAZ03250,NaN,NaN,[''],NaN,View Tags,NaN,NaN
1828416,8084,1056773,unz.com,bias,https://www.unz.com/gnxp/jim-manzi-on-expelled/,+\n\nAdvanced Search Options\n\nTitle/Subtitle...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Jim Manzi on Expelled,"Razib Khan, The Saker, John Derbyshire",NaN,[''],NaN,"9/11 Articles, American Pravda Articles, Race/...",NaN,NaN


# Deletion of unnecessary data
It was made to decrease the volume of dataset, otherwise it will be impossible to work with it.

In [7]:
train.drop('Unnamed: 0',axis=1, inplace=True)
train.drop('scraped_at',axis=1, inplace=True)
train.drop('inserted_at',axis=1, inplace=True)
train.drop('updated_at',axis=1, inplace=True)
train.drop('keywords',axis=1, inplace=True)
train.drop('summary',axis=1, inplace=True)
train.drop('source',axis=1, inplace=True)

In [8]:
train

,id,domain,type,url,content,title,authors,meta_keywords,meta_description,tags
0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,[''],THE UNIVERSE ceases to exist when we are not l...,NaN
1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"Unfortunately, he hasn’t yet attacked her for ...",Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",[''],NaN,NaN
4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“The time has come to cut off the tongues of t...,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",[''],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1828413,1056770,unz.com,bias,https://www.unz.com/gnxp/get-off-your-ass-and-...,I was recently directed to this panegyric on W...,Get off your ass and do this study: Introducto...,"Agnostic Gnxp, Razib Khan, John Harrison Sims,...",[''],NaN,"9/11 Articles, American Pravda Articles, Race/..."
1828414,1056771,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974STA...,Tor\n\nTor is an encrypted anonymising network...,Cable: 1974STATE092127,NaN,[''],NaN,View Tags
1828415,1056772,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974LAP...,Tor\n\nTor is an encrypted anonymising network...,Cable: 1974LAPAZ03250,NaN,[''],NaN,View Tags
1828416,1056773,unz.com,bias,https://www.unz.com/gnxp/jim-manzi-on-expelled/,+\n\nAdvanced Search Options\n\nTitle/Subtitle...,Jim Manzi on Expelled,"Razib Khan, The Saker, John Derbyshire",[''],NaN,"9/11 Articles, American Pravda Articles, Race/..."


# Cleaning the data
It is cleaned from stop words, symbols and made the lowercase

In [9]:
def clean_data(df):
    # Lower all characters
    df.content=df.content.apply(lambda sen: ' '.join(word.lower() for word in str(sen).split()))
    gc.collect()
    # Remove symbols
    df.content=df.content.apply(lambda sen: ''.join([word for word in sen if word.isalnum() or word==' ']))
    gc.collect()
    # Remove stop words
    sw=get_stop_words('english')
    df.content=df.content.apply(lambda sen: ' '.join([word for word in str(sen).split() if word not in sw]))
    gc.collect()
    print("here")
    print(df.content)
    return df


In [10]:
df = train

In [11]:
df= clean_data(df)

here
0          life is an illusion, at least on a quantum lev...
1          unfortunately, he hasn’t yet attacked her for ...
2          the los angeles police department has been den...
3          the white house has decided to quietly withdra...
4          “the time has come to cut off the tongues of t...
                                 ...                        
1828413    i was recently directed to this panegyric on w...
1828414    tor tor is an encrypted anonymising network th...
1828415    tor tor is an encrypted anonymising network th...
1828416    + advanced search options title/subtitle? auth...
1828417    a collection of interesting, important, and co...
Name: content, Length: 1828418, dtype: object
here
0          life is an illusion at least on a quantum leve...
1          unfortunately he hasnt yet attacked her for is...
2          the los angeles police department has been den...
3          the white house has decided to quietly withdra...
4          the time has come 

In [12]:
df

,id,domain,type,url,content,title,authors,meta_keywords,meta_description,tags
0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,life illusion least quantum level theory recen...,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,[''],THE UNIVERSE ceases to exist when we are not l...,NaN
1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,unfortunately hasnt yet attacked islamic terro...,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,los angeles police department denied 3 million...,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,white house decided quietly withdraw ties rama...,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",[''],NaN,NaN
4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,time come cut tongues support peace mourn cant...,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",[''],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1828413,1056770,unz.com,bias,https://www.unz.com/gnxp/get-off-your-ass-and-...,recently directed panegyric wikipedia claims e...,Get off your ass and do this study: Introducto...,"Agnostic Gnxp, Razib Khan, John Harrison Sims,...",[''],NaN,"9/11 Articles, American Pravda Articles, Race/..."
1828414,1056771,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974STA...,tor tor encrypted anonymising network makes ha...,Cable: 1974STATE092127,NaN,[''],NaN,View Tags
1828415,1056772,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974LAP...,tor tor encrypted anonymising network makes ha...,Cable: 1974LAPAZ03250,NaN,[''],NaN,View Tags
1828416,1056773,unz.com,bias,https://www.unz.com/gnxp/jim-manzi-on-expelled/,advanced search options titlesubtitle author r...,Jim Manzi on Expelled,"Razib Khan, The Saker, John Derbyshire",[''],NaN,"9/11 Articles, American Pravda Articles, Race/..."


In [13]:
gc.collect()

0

Created clean dataset

In [14]:
df.to_csv('cleaned.csv')

In [2]:
PATH_TO_TT = '/kaggle/working/'
df = pd.read_csv('/kaggle/input/cleanedfakenews/cleaned.csv')

df

/tmp/ipykernel_33/3503876693.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/cleanedfakenews/cleaned.csv')


,Unnamed: 0,id,domain,type,url,content,title,authors,meta_keywords,meta_description,tags
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,life illusion least quantum level theory recen...,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,[''],THE UNIVERSE ceases to exist when we are not l...,NaN
1,1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,unfortunately hasnt yet attacked islamic terro...,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
2,2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,los angeles police department denied 3 million...,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
3,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,white house decided quietly withdraw ties rama...,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",[''],NaN,NaN
4,4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,time come cut tongues support peace mourn cant...,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",[''],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1828413,1828413,1056770,unz.com,bias,https://www.unz.com/gnxp/get-off-your-ass-and-...,recently directed panegyric wikipedia claims e...,Get off your ass and do this study: Introducto...,"Agnostic Gnxp, Razib Khan, John Harrison Sims,...",[''],NaN,"9/11 Articles, American Pravda Articles, Race/..."
1828414,1828414,1056771,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974STA...,tor tor encrypted anonymising network makes ha...,Cable: 1974STATE092127,NaN,[''],NaN,View Tags
1828415,1828415,1056772,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974LAP...,tor tor encrypted anonymising network makes ha...,Cable: 1974LAPAZ03250,NaN,[''],NaN,View Tags
1828416,1828416,1056773,unz.com,bias,https://www.unz.com/gnxp/jim-manzi-on-expelled/,advanced search options titlesubtitle author r...,Jim Manzi on Expelled,"Razib Khan, The Saker, John Derbyshire",[''],NaN,"9/11 Articles, American Pravda Articles, Race/..."


In [3]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

# Tokenization and stemming with new dataset

In [4]:
def tokenize(txt):
    tokens = re.split('\W+', txt)
    text = [porter_stemmer.stem(word) for word in tokens]
    return text

In [5]:
df.content = df.content.apply(lambda x: tokenize(str(x)))
gc.collect()
df.head()

,Unnamed: 0,id,domain,type,url,content,title,authors,meta_keywords,meta_description,tags
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"[life, illus, least, quantum, level, theori, r...",Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,[''],THE UNIVERSE ceases to exist when we are not l...,NaN
1,1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"[unfortun, hasnt, yet, attack, islam, terroris...",Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
2,2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"[lo, angel, polic, depart, deni, 3, million, f...",Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
3,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,"[white, hous, decid, quietli, withdraw, tie, r...","MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",[''],NaN,NaN
4,4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,"[time, come, cut, tongu, support, peac, mourn,...","“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",[''],NaN,NaN


In [6]:
# PATH_TO_TT = '/kaggle/working/'
# df = pd.read_csv('/kaggle/input/firstfakenew/first.csv')

df.shape

(1828418, 11)

In [7]:
df

,Unnamed: 0,id,domain,type,url,content,title,authors,meta_keywords,meta_description,tags
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"[life, illus, least, quantum, level, theori, r...",Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,[''],THE UNIVERSE ceases to exist when we are not l...,NaN
1,1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"[unfortun, hasnt, yet, attack, islam, terroris...",Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
2,2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"[lo, angel, polic, depart, deni, 3, million, f...",Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
3,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,"[white, hous, decid, quietli, withdraw, tie, r...","MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",[''],NaN,NaN
4,4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,"[time, come, cut, tongu, support, peac, mourn,...","“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",[''],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1828413,1828413,1056770,unz.com,bias,https://www.unz.com/gnxp/get-off-your-ass-and-...,"[recent, direct, panegyr, wikipedia, claim, ed...",Get off your ass and do this study: Introducto...,"Agnostic Gnxp, Razib Khan, John Harrison Sims,...",[''],NaN,"9/11 Articles, American Pravda Articles, Race/..."
1828414,1828414,1056771,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974STA...,"[tor, tor, encrypt, anonymis, network, make, h...",Cable: 1974STATE092127,NaN,[''],NaN,View Tags
1828415,1828415,1056772,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974LAP...,"[tor, tor, encrypt, anonymis, network, make, h...",Cable: 1974LAPAZ03250,NaN,[''],NaN,View Tags
1828416,1828416,1056773,unz.com,bias,https://www.unz.com/gnxp/jim-manzi-on-expelled/,"[advanc, search, option, titlesubtitl, author,...",Jim Manzi on Expelled,"Razib Khan, The Saker, John Derbyshire",[''],NaN,"9/11 Articles, American Pravda Articles, Race/..."


In [ ]:
# def remove_quotes(txt):
#     print(txt)
#     print(txt[0])
#     return txt

In [ ]:
# df.content = df.content.apply(lambda x: remove_quotes(x.toArray()))
# df.head()

In [7]:
# from nltk.stem.porter import PorterStemmer
# porter_stemmer = PorterStemmer()

In [8]:
# def stemming(tokenized_text):
#     text = [porter_stemmer.stem(word) for word in tokenized_text]
#     return text

In [ ]:
# df.content = df.content.apply(lambda x: stemming(str(x)))
# df.head()

In [8]:
gc.collect()

0

In [9]:
df.to_csv('stemmed.csv')